In [117]:
import pulp
import pandas as pd

# 读取四个表格的数据
file_1 = '2023年的种植数据与销售量.xlsx'
file_2 = '各作物聚合后销售量与价格.xlsx'
file_3 = '各作物适合种植的地块类型与季别.xlsx'
file_4 = '乡村的现有耕地.xlsx'

# 加载表格
df1 = pd.read_excel(file_1)
df2 = pd.read_excel(file_2)
df3 = pd.read_excel(file_3)
df4 = pd.read_excel(file_4)




In [159]:
import random
# 由于计算量太大，这里简化为为每个地块随机选择9个作物
land_crop_mapping = {}
num_crops_to_select = 9

for land in land_types:
    if len(crops) > num_crops_to_select:
        selected_crops = random.sample(crops, num_crops_to_select)
    else:
        selected_crops = crops  # 如果作物少于10种，全部选中
    land_crop_mapping[land] = selected_crops

# 初始化决策变量和目标函数
decision_vars_limited = {}
objective_coeffs_limited = []

for land, selected_crops in land_crop_mapping.items():
    for crop in selected_crops:
        decision_vars_limited[(crop, land)] = 0
        
        land_type = df4[df4['地块名称'] == land]['地块类型'].values[0]
        yield_data = df1[(df1['作物名称_x'] == crop) & (df1['地块类型'] == land_type)]['亩产量/斤'].values
        price_data = df2[df2['作物名称'] == crop]['销售单价/(元/斤)'].values
        cost_data = df1[(df1['作物名称_x'] == crop) & (df1['地块类型'] == land_type)]['种植成本/(元/亩)'].values
        
        if len(yield_data) > 0 and len(price_data) > 0 and len(cost_data) > 0:
            yield_per_acre = yield_data[0]
            price_per_unit = price_data[0]
            cost_per_acre = cost_data[0]
            net_revenue = (yield_per_acre * price_per_unit) - cost_per_acre
            objective_coeffs_limited.append(net_revenue)
        else:
            objective_coeffs_limited.append(0)

objective_coeffs_limited = np.array(objective_coeffs_limited) * -1

# 重新构建约束条件
A_ub_limited = []
b_ub_limited = []

# 约束条件1：总面积约束
for land in land_types:
    constraint = np.zeros(len(decision_vars_limited))
    for i, (crop, land_name) in enumerate(decision_vars_limited.keys()):
        if land_name == land:
            constraint[i] = 1
    A_ub_limited.append(constraint)
    b_ub_limited.append(df4[df4['地块名称'] == land]['地块面积/亩'].values[0])

# 约束条件2：最小种植面积约束
for land in land_types:
    min_area = 0.1 * df4[df4['地块名称'] == land]['地块面积/亩'].values[0]
    for crop in land_crop_mapping[land]:
        constraint = np.zeros(len(decision_vars_limited))
        for i, (crop_name, land_name) in enumerate(decision_vars_limited.keys()):
            if crop_name == crop and land_name == land:
                constraint[i] = -1
        A_ub_limited.append(constraint)
        b_ub_limited.append(-min_area)

A_ub_limited = np.array(A_ub_limited)
b_ub_limited = np.array(b_ub_limited)

# 优化模型
result_limited = linprog(c=objective_coeffs_limited, A_ub=A_ub_limited, b_ub=b_ub_limited, method='highs')

# 检查优化结果
if result_limited.success:
    print("模型优化成功!")
else:
    print("优化失败:", result_limited.message)


随机抽取作物的模型优化成功!


In [160]:
# 构建优化结果的表格输出
if result_limited.success:
    optimal_areas_limited = result_limited.x
    solution_limited = {}
    
    for i, (crop, land) in enumerate(decision_vars_limited.keys()):
        if land not in solution_limited:
            solution_limited[land] = {}
        solution_limited[land][crop] = optimal_areas_limited[i]

    # 构建结果表格
    results_limited = pd.DataFrame(columns=['季别', '地块名'] + crops)
    
    for land, crop_areas in solution_limited.items():
        season_1 = {'季别': '第一季', '地块名': land}
        for crop in crops:
            season_1[crop] = crop_areas.get(crop, 0)  # 如果该作物不在该地块中，则面积为0
        
        results_limited = results_limited.append(season_1, ignore_index=True)

    # 显示结果表格
    results_limited.head()
else:
    results_limited = None
    print("优化失败，无法生成结果表格。")
    
results_limited.head()  # 输出结果表格的前5行


,季别,地块名,刀豆,包菜,南瓜,土豆,大白菜,大麦,小青菜,小麦,...,豇豆,辣椒,青椒,香菇,高粱,黄心菜,黄瓜,黄豆,黍子,黑豆
0,第一季,A1,0,0,0,8.0,0,0,0,0,...,0,0,0,8.0,0,0,0,0,8.0,0
1,第一季,A2,0,0,0,5.5,0,5.5,5.5,0,...,0,0,0,0.0,0,0,5.5,0,0.0,0
2,第一季,A3,0,0,0,3.5,0,0,0,0,...,0,3.5,0,0.0,0,0,0,3.5,3.5,3.5
3,第一季,A4,7.2,7.2,0,0.0,7.2,0,7.2,0,...,0,0,0,0.0,0,0,0,0,0.0,0
4,第一季,A5,0,0,0,0.0,0,6.8,6.8,0,...,0,0,0,0.0,6.8,0,0,0,0.0,0


In [ ]:
#以上输出的为第一季节，第二季只需要筛选出可以进行第二季的作物与地块进行重复输出即可
#具体代码可以查看b站up原创【不知名数学家小P】https://space.bilibili.com/435530921

In [162]:
results_limited.to_excel("2024年农作物种植方案.xlsx", index=False)